### Analyse et comprehension des fichiers

In [1]:
import matplotlib.pyplot as plt
from read_data import *
from utils import *
# from preprocess_utils import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
store_full_dataset_with_features(from_scratch=True, vectorize=True)

**distribution des données**

In [ ]:
train,n_train = get_train_data()
test,n_test = get_test_data()

In [ ]:
plt.figure(figsize=(10,5))
plt.hist(x=train['hindex'],bins=187);

**exploitation de model**

On est sur que le modèle applique softmax sur la représentation du mot. Nous ou choisit donc d'appliquer un autre régresseur

**Data for multiple regression model**

In [ ]:
X_train, y_train, X_test, y_test = get_numpy_data(n=174241)

In [ ]:
X_train_norm, X_test_norm = normalize(X_train, X_test)

**Random forest**

**XG_BOOST**

In [ ]:
model_xg = XGBRegressor(learning_rate=0.01, max_depth=5, objective="reg:linear", subsample=0.7, n_estimators=500)
model_xg.fit(X_train,y_train)
mod_preds = model_xg.predict(X_test)
mean_squared_error(y_test,mod_preds)

**Neural Network**

In [ ]:

from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model_nn = keras.Sequential()
model_nn.add(layers.Dense(64, kernel_initializer='uniform', input_shape=(10,)))
model_nn.add(layers.Activation('softmax'))

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)

model_nn.compile(loss='mse', optimizer=optimizer)

model_nn.fit(X_train,y_train, epochs=10, batch_size=5)
mod_nn_preds = model_nn.predict(X_test)
print(len(mod_nn_preds))
mean_squared_error(y_test,mod_nn_preds)

### CATBOOST

In [ ]:
from catboost import CatBoostRegressor
model_cat = CatBoostRegressor(verbose=False)
model_cat.fit(X_train, y_train)
mod_preds = model_cat.predict(X_test)
mean_squared_error(y_test,mod_preds)

In [ ]:
from catboost import cv, Pool
params = {
    "loss_function":"RMSE"
}
cv_dataset = Pool(data=X_train, label=y_train)
cv(cv_dataset, params=params, verbose=False)

**Light GBM**

In [ ]:
import lightgbm as lgb
d_train = lgb.Dataset(X_train, label = y_train)
params = {}
params['learning_rate'] = 0.33
params['num_iterations'] = 100
params['boosting_type'] = 'dart'
params['objective'] = 'regression'
params['metric'] = 'rmse'
params['sub_feature'] = 0.6
params['num_leaves'] = 50
params['min_data'] = 60
params['max_depth'] = 35
clf = lgb.train(params, d_train, 100)
y_pred=clf.predict(X_test)
mean_squared_error(y_test,y_pred)

**test linear regression**

In [ ]:
from sklearn import linear_model

reg = linear_model.TweedieRegressor().fit(X_train,y_train)

In [ ]:
reg_pred = reg.predict(X_test)

In [ ]:
mean_squared_error(y_test,reg_pred)

**test svm**

In [ ]:
from sklearn.svm import SVC

model_svm =SVC(kernel='sigmoid', C=2)

In [ ]:
model_svm.fit(X_train,y_train)

In [ ]:
svm_pred = model_svm.predict(X_test)

In [ ]:
mean_squared_error(y_test,svm_pred)

**Test CNN**

# Soumission

In [ ]:
from submit import submit
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
# from catboost import CatBoostRegressor

model_cat = CatBoostRegressor(verbose=False)
model_forest = RandomForestRegressor()
model_xg = XGBRegressor(learning_rate=0.01, max_depth=5, objective="reg:linear", subsample=0.7, n_estimators=500)

submit(model_cat)

In [ ]:
import lightgbm as lgb

X_train, y_train, X_test, y_test = get_submission_data()
X_train, X_test = normalize(X_train, X_test)


d_train = lgb.Dataset(X_train, label = y_train)
params = {}
params['learning_rate'] = 0.10
params['num_iterations'] = 500
params['boosting_type'] = 'dart'
params['objective'] = 'regression'
params['metric'] = 'rmse'
params['sub_feature'] = 0.6
params['num_leaves'] = 50
params['min_data'] = 60
params['max_depth'] = 35
clf = lgb.train(params, d_train, 100)
y_pred=clf.predict(X_test)


test, _ = get_test_data()
test["hindex"] = y_pred
submission = test[["author", "hindex"]]
submission.to_csv("../tmp/submission.csv", index=None)